<a href="https://colab.research.google.com/github/bwbayu/TalentMatch/blob/development/model/praproses/resume_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wordninja num2words inflect datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=4b01abb7545a734c76cb8aedead674e21f95ea103436f3e0f5bc61f56af4faa0
  Stored in directory: /root/.cache/pip

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import wordninja
from num2words import num2words
import inflect
from datasets import load_dataset
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import spacy

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
p = inflect.engine()
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Kaggle Dataset : https://www.kaggle.com/datasets/gauravduttakiit/resume-dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Dataset/Compfest16_AIC/Raw/UpdatedResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [ ]:
df.shape

(962, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  962 non-null    object
 1   Resume    962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [ ]:
df = df.drop_duplicates(subset=['Resume'], keep='first')

In [ ]:
def preprocessing_data(text, idx):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove link
    text = re.sub(r'\b(?:https?://)?(?:www\.)?[a-zA-Z0-9-]+\.(?:com|org|net|edu|gov|co|co\.id|info|biz|us|ca|uk|de|jp)\S*\s*', '', text) # remove link
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text) # remove email
    text = re.sub(r'\+?\d[\d -]{8,}\d', '', text) # remove phone number
    text = re.sub(r'\b\d{1,4}[/-]\d{1,2}[/-]\d{1,4}\b', '', text) # remove date -> 01/12/2000
    text = re.sub(r'\b[A-Za-z]+\s\d{4}\b', '', text) # remove date -> May 2013
    text = re.sub(r'[^\w\s]', ' ', text) # punctuation
    text = text.replace("exprience", "experience")
    text = text.replace("matelabs", "matlab")
    text = text.replace("maharashtra", " ")
    text = text.replace("monthscompany", " ")
    text = text.replace("-", " ")

    # convert number to word with the following requirement
    pattern = r'\b(\d+)\s+(year|years|months|month|experience|position|skill)\b'

    def replace_match(match):
        number = match.group(1)
        word = p.number_to_words(number)
        return f"{word} {match.group(2)}"

    # Gantikan angka relevan dengan kata
    text = re.sub(pattern, replace_match, text, flags=re.IGNORECASE)

    # Hapus angka yang tidak relevan
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b\w\b', '', text) # remove single character
    text = re.sub(r'[^\w\s]', ' ', text) # punctuation

    # remove stopword and lemmatization
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)

    # remove word that related to location
    doc = nlp(text)
    tokens = [token.text for token in doc if token.ent_type_ != 'GPE']
    text = ' '.join(tokens)

    # # remove verb
    # doc = nlp(text)
    # non_verbs = [token.text for token in doc if token.pos_ not in ["VERB"]]
    # text = ' '.join(non_verbs)

    print(f"Index: {idx}, Preprocessed Text : {' '.join(tokens[:5])}")
    return text


In [ ]:
df['clean_data'] = df.apply(lambda row: preprocessing_data(row['Resume'], row.name), axis=1)

Index: 0, Preprocessed Text : skill programming language python panda
Index: 1, Preprocessed Text : education detail uit rgpv data
Index: 2, Preprocessed Text : area interest deep learning control
Index: 3, Preprocessed Text : skill python sap hana tableau
Index: 4, Preprocessed Text : education detail mca ymcaust faridabad
Index: 5, Preprocessed Text : skill basic iot python matlab
Index: 6, Preprocessed Text : skill python tableau data visualization
Index: 7, Preprocessed Text : education detail tech rayat bahra
Index: 8, Preprocessed Text : personal skill ability quickly grasp
Index: 9, Preprocessed Text : expertise data quantitative analysis decision
Index: 40, Preprocessed Text : technical skill typewriting tora spsseducation
Index: 41, Preprocessed Text : skill window xp m office
Index: 42, Preprocessed Text : education detail ba mumbai university
Index: 43, Preprocessed Text : education detail economics chennai tamil
Index: 45, Preprocessed Text : education detail bba lovely pro

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 0 to 898
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Category    166 non-null    object
 1   Resume      166 non-null    object
 2   clean_data  166 non-null    object
dtypes: object(3)
memory usage: 5.2+ KB


In [ ]:
df['text_length'] = df['clean_data'].apply(lambda x: len(x.split()))
print(df['text_length'].describe())

count     166.000000
mean      294.795181
std       256.579092
min        14.000000
25%       108.000000
50%       229.000000
75%       404.000000
max      1411.000000
Name: text_length, dtype: float64


In [ ]:
df[df['text_length'] > 405].count()

,0
Category,41
Resume,41
clean_data,41
text_length,41


In [ ]:
all_words = ' '.join(df['clean_data']).split()
word_freq = Counter(all_words)
# print(word_freq.most_common()[-1000:-5000:-1])
# print(word_freq.most_common(100))

In [ ]:
# Tentukan ambang batas untuk menghapus kata-kata yang jarang muncul
# Misalnya, kita akan menghapus kata-kata yang muncul kurang dari 2 kali
threshold = 2
rare_words = [word for word, freq in word_freq.items() if freq <= threshold]

# Fungsi untuk menghapus kata-kata yang jarang muncul dari teks
def remove_rare_words(text, rare_words):
    return ' '.join([word for word in text.split() if word not in rare_words])

# Terapkan fungsi ini ke setiap teks dalam data
cleaned_texts = [remove_rare_words(text, rare_words) for text in df['clean_data']]
df = pd.DataFrame(cleaned_texts, columns=['clean_data'])

In [ ]:
df['text_length'] = df['clean_data'].apply(lambda x: len(x.split()))
print(df['text_length'].describe())

count     166.000000
mean      294.795181
std       256.579092
min        14.000000
25%       108.000000
50%       229.000000
75%       404.000000
max      1411.000000
Name: text_length, dtype: float64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 0 to 898
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Category     166 non-null    object
 1   Resume       166 non-null    object
 2   clean_data   166 non-null    object
 3   text_length  166 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 6.5+ KB


In [ ]:
df.to_csv('/content/drive/MyDrive/Dataset/Compfest16_AIC/dataset_resume.csv', index=False)

# Hugging Face Resume Job : https://huggingface.co/datasets/cnamuangtoun/resume-job-description-fit

In [ ]:
ds = load_dataset("cnamuangtoun/resume-job-description-fit")

df_train = pd.DataFrame(ds['train'])
df_test = pd.DataFrame(ds['test'])

df_train = df_train.drop(columns=['label'])
df_test = df_test.drop(columns=['label'])

df_cv = pd.concat([df_train['resume_text'], df_test['resume_text']], ignore_index=True)
df_jd = pd.concat([df_train['job_description_text'], df_test['job_description_text']], ignore_index=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [ ]:
df_cv.info()

<class 'pandas.core.series.Series'>
RangeIndex: 8000 entries, 0 to 7999
Series name: resume_text
Non-Null Count  Dtype 
--------------  ----- 
8000 non-null   object
dtypes: object(1)
memory usage: 62.6+ KB


In [ ]:
train_duplicates = df_cv[df_cv.duplicated()].count()
print(train_duplicates)

7357


In [ ]:
df_cv = df_cv.drop_duplicates(keep='first')

In [ ]:
df_cv.info()

<class 'pandas.core.series.Series'>
Index: 643 entries, 0 to 6277
Series name: resume_text
Non-Null Count  Dtype 
--------------  ----- 
643 non-null    object
dtypes: object(1)
memory usage: 10.0+ KB


In [ ]:
def preprocessing_data1(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove link
    text = re.sub(r'\b(?:https?://)?(?:www\.)?[a-zA-Z0-9-]+\.(?:com|org|net|edu|gov|co|co\.id|info|biz|us|ca|uk|de|jp)\S*\s*', '', text) # remove link
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text) # remove email
    text = re.sub(r'\+?\d[\d -]{8,}\d', '', text) # remove phone number
    text = re.sub(r'\b\d{1,4}[/-]\d{1,2}[/-]\d{1,4}\b', '', text) # remove date -> 01/12/2000
    text = re.sub(r'\b[A-Za-z]+\s\d{4}\b', '', text) # remove date -> May 2013
    text = re.sub(r'[^\w\s]', ' ', text) # punctuation

    # convert number to word with the following requirement
    pattern = r'\b(\d+)\s+(year|years|months|month|experience|position|skill)\b'

    def replace_match(match):
        number = match.group(1)
        word = p.number_to_words(number)
        return f"{word} {match.group(2)}"

    # Gantikan angka relevan dengan kata
    text = re.sub(pattern, replace_match, text, flags=re.IGNORECASE)

    # Hapus angka yang tidak relevan
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b\w\b', '', text) # remove single character

    # remove stopword and lemmatization
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    preprocessed_text = ' '.join(tokens)
    # print(f"Index: {idx}, Preprocessed Text : {' '.join(tokens[:5])}")
    return preprocessed_text


# Hugging Face Resume Job : https://huggingface.co/datasets/InferencePrince555/Resume-Dataset?row=0

In [ ]:
ds1 = load_dataset("InferencePrince555/Resume-Dataset")
df_cv1 = pd.DataFrame(ds1['train'])
df_cv1 = df_cv1.drop(columns=['input'])
df_cv1.head(20)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32481 [00:00<?, ? examples/s]

,instruction,Resume_test
0,Generate a Resume for a Accountant Job,ACCOUNTANT Professional Summary Results orient...
1,Generate a Resume for a Accountant Job,STAFF ACCOUNTANT Summary Flexible Accountant w...
2,Generate a Resume for a Accountant Job,STAFF ACCOUNTANT Summary Highly analytical and...
3,Generate a Resume for a Accountant Job,SENIOR ACCOUNTANT Summary A highly competent m...
4,Generate a Resume for a Accountant Job,SENIOR ACCOUNTANT Summary 11 years experience ...
5,Generate a Resume for a Accountant Job,FINANCIAL ACCOUNTANT Summary CPA Financial Acc...
6,Generate a Resume for a Accountant Job,CORPORATE ACCOUNTANT Summary Over 15 years of ...
7,Generate a Resume for a Accountant Job,ACCOUNTANT Professional Summary Current Accoun...
8,Generate a Resume for a Accountant Job,ACCOUNTANT Summary Innovative and energetic Ac...
9,Generate a Resume for a Accountant Job,ACCOUNTANT Highlights Microsoft Office Interme...


In [ ]:
train_duplicates = df_cv1[df_cv1['Resume_test'].duplicated()].count()
print(train_duplicates)

instruction    818
Resume_test    818
dtype: int64


In [ ]:
df_cv1 = df_cv1.drop_duplicates(keep='first')
df_cv1 = df_cv1.dropna(subset=['Resume_test'])

In [ ]:
def preprocess_job_title(text):
    cleaned_text = re.sub(r'Generate a Resume for a\s+|\s+Job', '', text, flags=re.IGNORECASE)

    cleaned_text = cleaned_text.lower()

    return cleaned_text

In [ ]:
df_cv1['category'] = df_cv1['instruction'].apply(preprocess_job_title)

<ipython-input-23-4955fe9b6f1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cv1['category'] = df_cv1['instruction'].apply(preprocess_job_title)


In [ ]:
df_cv1 = df_cv1.drop(columns=['instruction'])
df_cv1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31680 entries, 0 to 32480
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Resume_test  31680 non-null  object
 1   category     31680 non-null  object
dtypes: object(2)
memory usage: 742.5+ KB


In [ ]:
df_cv1['category'] = df_cv1['category'].replace('web designing', 'UI/UX Designer')
df_cv1['category'] = df_cv1['category'].replace('database', 'database administrator')

In [ ]:
df_cv1['category'].value_counts()

,count
category,
software developer,5827
systems administrator,4181
project manager,3527
web developer,3466
database administrator,2795
java developer,2431
python developer,2317
network administrator,2260
security analyst,2259


In [ ]:
def preprocessing_data1(text, idx):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # remove link
    text = re.sub(r'\b(?:https?://)?(?:www\.)?[a-zA-Z0-9-]+\.(?:com|org|net|edu|gov|co|co\.id|info|biz|us|ca|uk|de|jp)\S*\s*', '', text) # remove link
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text) # remove email
    text = re.sub(r'\+?\d[\d -]{8,}\d', '', text) # remove phone number
    text = re.sub(r'\b\d{1,4}[/-]\d{1,2}[/-]\d{1,4}\b', '', text) # remove date -> 01/12/2000
    text = re.sub(r'\b[A-Za-z]+\s\d{4}\b', '', text) # remove date -> May 2013
    text = re.sub(r'[^\w\s]', ' ', text) # punctuation

    # remove text from this pattern
    pattern = r'.*?\bsummary\b'
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # convert number to word with the following requirement
    pattern = r'\b(\d+)\s+(year|years|months|month|experience|position|skill)\b'

    def replace_match(match):
        number = match.group(1)
        word = p.number_to_words(number)
        return f"{word} {match.group(2)}"

    # Gantikan angka relevan dengan kata
    text = re.sub(pattern, replace_match, text, flags=re.IGNORECASE)

    # Hapus angka yang tidak relevan
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b\w\b', '', text) # remove single character
    text = re.sub(r'[^\w\s]', ' ', text) # punctuation

    # remove stopword and lemmatization
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)

    # # remove word that related to location, date, and cardinal
    # doc = nlp(text)
    # tokens = [token.text for token in doc if token.ent_type_ != 'GPE' and token.ent_type_ != 'DATE' and token.ent_type_ != 'CARDINAL' ]
    # text = ' '.join(tokens)

    # # remove verb
    # doc = nlp(text)
    # non_verbs = [token.text for token in doc if token.pos_ not in ["VERB"]]
    # text = ' '.join(non_verbs)

    print(f"Index: {idx}, Preprocessed Text : {' '.join(tokens[:5])}")
    return text


In [ ]:
df_cv1['clean_data'] = df_cv1.apply(lambda row: preprocessing_data1(row['Resume_test'], row.name), axis=1)

Streaming output truncated to the last 5000 lines.
Index: 14506, Preprocessed Text : senior java developer senior span
Index: 14507, Preprocessed Text : cyber security analyst cyber span
Index: 14508, Preprocessed Text : cloud devops engineer cloud devops
Index: 14509, Preprocessed Text : python developer span lpythonspan span
Index: 14510, Preprocessed Text : freelance designer production artist april
Index: 14511, Preprocessed Text : lead python developer lead span
Index: 14512, Preprocessed Text : database administrator consumer lending department
Index: 14513, Preprocessed Text : sr middleware administratorweb system engineer
Index: 14514, Preprocessed Text : python developer span lpythonspan span
Index: 14515, Preprocessed Text : language java j2ee sql javascript
Index: 14516, Preprocessed Text : education detail diploma computer science
Index: 14517, Preprocessed Text : resolve technical business incident independently
Index: 14518, Preprocessed Text : report cyber security opera

In [ ]:
df_cv1 = df_cv1.drop(columns=['Resume_test'])
df['text_length'] = df['clean_data'].apply(lambda x: len(x.split()))
print(df['text_length'].describe())

In [ ]:
df_cv1.head()

In [ ]:
df_cv1.to_csv("/content/drive/MyDrive/Dataset/Compfest16_AIC/resume30k.csv")